In [1]:
import os
import pandas as pd
import mysql.connector
from mysql.connector import errorcode

In [2]:
# connection arguments

DB_NAME = 'nutrition'

config = {
    'user': os.environ.get('DB_USER'),
    'password': os.environ.get('DB_PASS'),
    'host': os.environ.get('DB_HOST'),
    'allow_local_infile': True, 
    'raise_on_warnings': True}

## Connection

In [3]:
# connection

try:
    cnx = mysql.connector.connect(**config)
    cursor = cnx.cursor()
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
        cursor.close()
        cnx.close()

In [4]:
cursor.execute(f"DROP DATABASE IF EXISTS {DB_NAME}")

## Create database

In [5]:
# create database

def create_database(cursor):
    try:
        cursor.execute(f"CREATE DATABASE {DB_NAME} DEFAULT CHARACTER SET 'utf8mb4'")
    except mysql.connector.Error as err:
        print(f"Failed creating database: {err}")
        exit(1)

try:
    cursor.execute(f"USE {DB_NAME}")
except mysql.connector.Error as err:
    print(f"Database {DB_NAME} does not exists.")
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor)
        print(f"Database {DB_NAME} created successfully.")
        cnx.database = DB_NAME
    else:
        print(err)
        exit(1)

Database nutrition does not exists.
Database nutrition created successfully.


## Create tables

In [6]:
# creating Tables

table_name = 'dispo_alim'

table_description = """
CREATE TABLE IF NOT EXISTS `dispo_alim` (
    `code_pays` int NOT NULL,
    `pays` text,
    `annee` int DEFAULT NULL,
    `code_produit` int NOT NULL,
    `produit` text,
    `origin` text,
    `dispo_alim_kcal_p_j` double DEFAULT NULL,
    `dispo_prot` double DEFAULT NULL,
    `dispo_mat_gr` double DEFAULT NULL,
    `dispo_alim_tonnes` double DEFAULT NULL,
    PRIMARY KEY (`code_pays`,`code_produit`)) DEFAULT CHARSET=utf8mb4;
"""

try:
    print(f"Creating table {table_name} : ", end = '')
    cursor.execute(table_description)
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
        print("already exists.")
    else:
        print(err.msg)
else:
    print("OK.")

Creating table dispo_alim : OK.


In [7]:
# creating Tables

table_name = 'equilibre_prod'

table_description = """
CREATE TABLE IF NOT EXISTS `equilibre_prod` (
    `code_pays` int NOT NULL,
    `pays` text,
    `annee` int DEFAULT NULL,
    `code_produit` int NOT NULL,
    `produit` text,
    `origin` text,
    `dispo_int` double DEFAULT NULL,
    `alim_ani` double DEFAULT NULL,
    `semences` double DEFAULT NULL,
    `pertes` double DEFAULT NULL,
    `nourriture` double DEFAULT NULL,
    `transfo` double DEFAULT NULL,
    `autres_utilisations` double DEFAULT NULL,
    PRIMARY KEY (`code_pays`,`code_produit`)) DEFAULT CHARSET=utf8mb4;
"""

try:
    print(f"Creating table {table_name} : ", end = '')
    cursor.execute(table_description)
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
        print("already exists.")
    else:
        print(err.msg)
else:
    print("OK.")

Creating table equilibre_prod : OK.


In [8]:
# creating Tables

table_name = 'population'

table_description = """
CREATE TABLE IF NOT EXISTS `population` (
    `code_pays` int NOT NULL,
    `pays` text,
    `annee` int DEFAULT NULL,
    `population` int DEFAULT NULL,
    PRIMARY KEY (`code_pays`)) DEFAULT CHARSET=utf8mb4;
"""

try:
    print(f"Creating table {table_name} : ", end = '')
    cursor.execute(table_description)
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
        print("already exists.")
    else:
        print(err.msg)
else:
    print("OK.")

Creating table population : OK.


In [9]:
# creating Tables

table_name = 'sous_nutrition'

table_description = """
CREATE TABLE IF NOT EXISTS `sous_nutrition` (
    `code_pays` int NOT NULL,
    `pays` text,
    `annee` int DEFAULT NULL,
    `nb_personnes` double DEFAULT NULL,
    PRIMARY KEY (`code_pays`)) DEFAULT CHARSET=utf8mb4;
"""

try:
    print(f"Creating table {table_name} : ", end = '')
    cursor.execute(table_description)
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
        print("already exists.")
    else:
        print(err.msg)
else:
    print("OK.")

Creating table sous_nutrition : OK.


## Insert data

In [10]:
# Inserting Data

add_population = """
LOAD DATA LOCAL INFILE 'population.csv'
REPLACE
INTO TABLE `population`
FIELDS TERMINATED BY ';'
LINES TERMINATED BY '\n'
IGNORE 1 LINES;
"""
cursor.execute(add_population)

# When you use a transactional storage engine such as InnoDB (the default in MySQL 5.5 and higher), 
# you must commit the data after a sequence of INSERT, DELETE, and UPDATE statements.

cnx.commit()

In [11]:
# Inserting Data

add_sous_nutrition = """
LOAD DATA LOCAL INFILE 'sous_nutrition.csv'
REPLACE
INTO TABLE `sous_nutrition`
FIELDS TERMINATED BY ';'
LINES TERMINATED BY '\n'
IGNORE 1 LINES;
"""
cursor.execute(add_sous_nutrition)

# Make sure data is committed to the database
cnx.commit()

In [12]:
# Inserting Data

add_dispo_alim = """
LOAD DATA LOCAL INFILE 'dispo_alim.csv'
REPLACE
INTO TABLE `dispo_alim`
FIELDS TERMINATED BY ';'
LINES TERMINATED BY '\n'
IGNORE 1 LINES;
"""
cursor.execute(add_dispo_alim)

# Make sure data is committed to the database
cnx.commit()

In [13]:
# Inserting Data

add_equilibre_prod = """
LOAD DATA LOCAL INFILE 'equilibre_prod.csv'
REPLACE
INTO TABLE `equilibre_prod`
FIELDS TERMINATED BY ';'
LINES TERMINATED BY '\n'
IGNORE 1 LINES;
"""
cursor.execute(add_equilibre_prod)

# Make sure data is committed to the database
cnx.commit()

## Requêtes

In [14]:
# Les 10 pays ayant le plus haut ratio disponibilité alimentaire par habitant en Kcal par habitant.

query = """
SELECT pays AS Pays, ROUND(SUM(dispo_alim_kcal_p_j)) AS Dispo_kcal
FROM dispo_alim
GROUP BY pays
ORDER BY Dispo_kcal DESC
LIMIT 10;
"""

cursor.execute(query)
table_rows = cursor.fetchall()
df = pd.DataFrame(table_rows)

### Disponibilité alimentaire par habitant en Kcal, les 10 plus grands ratios.

In [15]:
df.columns = ['Pays', 'Kcal/personne/jour']
df.set_index('Pays', inplace=True)
df

,Kcal/personne/jour
Pays,
Autriche,3770.0
Belgique,3737.0
Turquie,3708.0
États-Unis d'Amérique,3682.0
Israël,3610.0
Irlande,3602.0
Italie,3578.0
Luxembourg,3540.0
Égypte,3518.0


In [16]:
# Les 10 pays ayant le plus faible ratio disponibilité alimentaire par habitant en Kcal par habitant.

query = """
SELECT pays AS Pays, ROUND(SUM(dispo_alim_kcal_p_j)) AS Dispo_kcal
FROM dispo_alim
GROUP BY pays
ORDER BY Dispo_kcal
LIMIT 10;
"""

cursor.execute(query)
table_rows = cursor.fetchall()
df = pd.DataFrame(table_rows)

### Disponibilité alimentaire par habitant en Kcal, les 10 plus faibles ratios.

In [17]:
df.columns = ['Pays', 'Kcal/personne/jour']
df.set_index('Pays', inplace=True)
df

,Kcal/personne/jour
Pays,
République centrafricaine,1879.0
Zambie,1924.0
Madagascar,2056.0
Afghanistan,2087.0
Haïti,2089.0
République populaire démocratique de Corée,2093.0
Tchad,2109.0
Zimbabwe,2113.0
Ouganda,2126.0


In [18]:
# La quantité totale (en tonnes) de produits perdus par pays.

query = """
SELECT pays AS Pays, SUM(pertes)*1000 AS Pertes_tonnes
FROM equilibre_prod -- valeurs en milliers de tonnes
GROUP BY pays
ORDER BY Pertes_tonnes DESC;
"""

cursor.execute(query)
table_rows = cursor.fetchall()
df = pd.DataFrame(table_rows)

### Quantité totale de produits perdus par pays.

In [19]:
df.columns = ['Pays', 'Pertes en tonnes']
df.set_index('Pays', inplace=True)
df.head(24)

,Pertes en tonnes
Pays,
Chine,90358000.0
Brésil,75914000.0
Inde,55930000.0
Nigéria,19854000.0
Indonésie,13081000.0
Turquie,12036000.0
Mexique,8289000.0
Égypte,7608000.0
Ghana,7442000.0


In [20]:
# Les 10 pays pour lesquels la proportion de personnes sous-alimentées est la plus forte.

query = """
SELECT population.pays AS Pays, ROUND((sous_nutrition.nb_personnes/population.population)*100, 2) AS ratio_sous_nutrition
FROM population
INNER JOIN sous_nutrition
-- ON population.code_pays = sous_nutrition.code_pays
USING (code_pays)
ORDER BY ratio_sous_nutrition DESC
LIMIT 10;
"""

cursor.execute(query)
table_rows = cursor.fetchall()
df = pd.DataFrame(table_rows)

### Les 10 pays pour lesquels la proportion de personnes sous-alimentées est la plus forte.

In [21]:
df.columns = ['Pays', 'Proportion de personnes sous-alimentées (%)']
df.set_index('Pays', inplace=True)
df

,Proportion de personnes sous-alimentées (%)
Pays,
Haïti,50.40
Zambie,48.15
Zimbabwe,46.64
République centrafricaine,43.33
République populaire démocratique de Corée,42.58
Congo,40.47
Tchad,38.21
Angola,37.72
Libéria,37.26


In [22]:
# -- Les 10 produits pour lesquels le ratio Autres utilisations/Disponibilité intérieure est le plus élevé.

query = """
SELECT produit AS Produit, ROUND((SUM(autres_utilisations)/SUM(dispo_int))*100, 0) AS ratio_autres_util
FROM equilibre_prod
GROUP BY produit
ORDER BY ratio_autres_util DESC
LIMIT 10;
"""

cursor.execute(query)
table_rows = cursor.fetchall()
df = pd.DataFrame(table_rows)

### Les 10 produits pour lesquels le ratio 'Autres utilisations' sur 'Disponibilité intérieure' est le plus élevé.

In [23]:
df.columns = ['Produits', 'Autres utilisations/Disponibilité intérieure (%)']
df.set_index('Produits', inplace=True)
df

,Autres utilisations/Disponibilité intérieure (%)
Produits,
"Alcool, non Comestible",100.0
Huil Plantes Oleif Autr,76.0
Huile de Palmistes,70.0
Huile de Palme,70.0
Girofles,65.0
Huile de Colza&Moutarde,55.0
Graisses Animales Crue,47.0
Huiles de Poissons,45.0
Huile de Soja,42.0


In [24]:
cursor.close()
cnx.close()